In [72]:
df = pd.read_csv('../../../input/treino.csv')

In [73]:
df, _ = dataPipeline(df)

Dataframe carregado!
Colunas 'item' e 'semana_comercial' dropadas!
Vendas Guide Shop (GS) dropadas!
Vendas negativas alteradas para zero!
Variaveis categoricas transformadas em numéricas!
coluna 'data_semana_comercial' transformada em datetime!
Dataset de treino para lojas web criado
Dataset de treino para lojas fisicas criado


In [74]:
#Feature engineering
df['is_month_start'] = df.data_semana_comercial.dt.is_month_start.astype(int)
df['is_month_end'] = df.data_semana_comercial.dt.is_month_end.astype(int)
df['month'] = df.data_semana_comercial.dt.month
df["is_wknd"] = df.data_semana_comercial.dt.weekday // 4

In [75]:
def random_noise(dataframe):
    return np.random.normal(scale=1.5, size=(len(dataframe),))

In [76]:
df = pd.get_dummies(df, columns=['cod_loja'])

In [77]:
lgb_params = {'num_leaves': 10,
              'learning_rate': 0.02,
              'feature_fraction': 0.8,
              'max_depth': 5,
              'verbose': 0,
              'num_boost_round': 10000,
              'early_stopping_rounds': 200,
              'nthread': -1}

In [78]:
df = df[['data_semana_comercial', 'is_month_start', 'is_month_end', 'month', 'is_wknd', 'cod_loja_406', 'cod_loja_407', 'venda']]

In [79]:
df.head()

,data_semana_comercial,is_month_start,is_month_end,month,is_wknd,cod_loja_406,cod_loja_407,venda
4,2022-06-05,0,0,6,1,False,True,188
20,2021-05-23,0,0,5,1,False,True,136
58,2021-05-30,0,0,5,1,True,False,926
150,2022-04-03,0,0,4,1,False,True,152
253,2022-07-24,0,0,7,1,False,True,33


In [80]:
x = df.iloc[:, 1:7]
y = df.iloc[:, 7]

In [81]:
cols = ['is_month_start', 'is_month_end', 'month', 'is_wknd', 'cod_loja_406', 'cod_loja_407']

In [82]:
def smape(preds, target):
    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds - target)
    denom = np.abs(preds) + np.abs(target)
    smape_val = (200 * np.sum(num / denom)) / n
    return smape_val


def lgbm_smape(preds, train_data):
    labels = train_data.get_label()
    smape_val = smape(preds, labels)
    return 'SMAPE', smape_val, False

In [83]:
tss = TimeSeriesSplit(n_splits=5)
df = df.sort_index()


fold = 0
preds_loja = []
scores_loja = []
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]


    TARGET = 'venda'

    X_train = train[cols]
    y_train = train[TARGET]

    X_test = test[cols]
    y_test = test[TARGET]
    lgbtrain = lgb.Dataset(data=X_train, label=y_train, feature_name=cols)
    lgbval = lgb.Dataset(data=X_test, label=y_test, reference=lgbtrain, feature_name=cols)
    model = lgb.train(lgb_params, lgbtrain,
                      valid_sets=[lgbtrain, lgbval],
                  num_boost_round=lgb_params['num_boost_round'],
                  early_stopping_rounds=lgb_params['early_stopping_rounds'],
                  feval=lgbm_smape,
                  verbose_eval=100)

    y_pred_val = model.predict(X_test, num_iteration=model.best_iteration)

    preds_loja.append(y_pred_val)
    score = smape(y_pred_val, y_test)
    scores_loja.append(score)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000038 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 200 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp